# Getting data (read CSV), clean and export to CSV/into DB

### Step 01
In order to **get data** and store it into a Pandas dataframe the following code needs to be provided with:
* `file` to be imported
* `room` name of the room, e.g. `BA` for bath (Bad)
* `val` value to be imported, e.g. `Temp` for temperature

### Step 02
Data **cleaning** is done via:
* removing duplicates (first occurrence is kept)
* filling gaps by method 'fill forward'

The cleaned dataframe is per default **saved to a CSV**. If the dataframe shall be saved to QuestDB, also the following parameters are important: \
`step_db` to be set to `True`

> Communication with QuestDB with (preset) parameters: \
> `HOST` and \
> `PORT` to be able to communicate with the db

In [37]:
from questdb.ingress import Sender, IngressError
import sys
import csv
import pandas as pd
import numpy as np
from datetime import date

# global variables
g_len_list = 10 #variable for dataframe head or tail method
debug = 0

###### INPUTS
path = 'S:/Andreas/FH/Technikum/BA/'  #including slash at the end!
get = '10_OwnData/'
put = '40_Prep/'

filename = 'Windgeschwindigkeit - 221_Garten_10mins.csv'
ROOM = 'OG'
VAL = 'W'
delimiter = ','
###### ######

# logic for importing wind or temperatures
class Import:
    def __init__(self):
        step_db = False           #'False' do not import into QuestDB (first CAPITAL letter!!!)
        self.process(step_db) #call function
        
    def process(self, step_db):
        global g_len_list
        # 01 import data from csv out of Loxone
        #def read():
        data = csv.DictReader(open(path+get+filename, encoding="ISO8859"), delimiter=delimiter)
        df = pd.DataFrame(data)

        #create columns to import  ## creating new column at the end: df.loc[:, "End"] = ...
        if 'Datum' in df.columns:
            bool = True
            print (f'> Column "Datum" or "Zeit" in csv found, applying appropriate pre-import steps')
            df['UTC'] = pd.to_datetime(df['Datum'] + ' ' + df['Zeit'])
            df[ROOM+'_'+VAL] = pd.to_numeric(df.iloc[:, 2])  #pd.to_numeric(df['Temp bei Tür'])
        else:
            bool = False
            print (f'> Column "Datum" or "Zeit" in csv NOT found, applying appropriate pre-import steps')
            df['UTC'] = pd.to_datetime(df.iloc[:, 0])  #, utc=False, dtype='datetime64[ns, UTC-05:00]','
            df[ROOM+'_'+VAL] = pd.to_numeric(df.iloc[:, 1])
        
        print (f'> {len(df.index)} lines into DF imported')

        # 02 check for duplicates, list and clean them
        ids = df["UTC"]
        if debug:
            print(df[ids.isin(ids[ids.duplicated()])].sort_values("UTC").tail(g_len_list))
        
        # drop duplicate lines (if any)
        df = df.drop_duplicates(subset=['UTC']) #important: as same timestamp might have different values we search for duplicates only in UTC (and take the first row as the 'right' one) 
        ##print (df.head(5))
        if (bool):
            df.iloc[:, [1,2,3,0,4]] #changing column order
            df_short = df.iloc[:, 3:5] #only use the new columns  ##iloc: all the rows, only columns 3 to x
        else:
            ##df.iloc[:, [1,2,0,3]]
            ##print (df.head(5))
            df_short = df.iloc[:, [0, 3]]
        print (f'> {len(df_short.index)} lines in DF ok (after duplicates)')
        
        if debug:
            print (df_short.head(5))
            #df_short.dtypes
            #print(df_short.info())
        
        # 03 add data to the 10mins timestamp list
        df_norm = self.time_normalize(df_short)
        
        # 04 fill gaps and create flag column
        fl = ROOM+'_'+VAL+'_FLAG'
        df_norm[fl] = np.where(df_norm[ROOM+'_'+VAL].isna(), 500, 100)  #100 = original and ok, 500 is automatically added (because of fill forward)
        df_norm[fl] = pd.to_numeric(df_norm[fl])
        
        df_fill = self.gaps_fill(df_norm)
        
        # 05 if selected, send to DB - else export to csv
        if (step_db):
            self.db_import(df_fill) #call function
        else:
            self.csv_export(df_fill, ROOM, VAL) #call function
            
    def fill_gap_in_col(self, col, method):
        """
        Fills true gap in series.
        """
        colf = col.copy()
        first_idx = colf.first_valid_index()
        last_idx = colf.last_valid_index()
        if (method == 'ffill'):
            colf.loc[first_idx:last_idx] = colf.loc[first_idx:last_idx].ffill()
        else:
            colf.loc[first_idx:last_idx] = colf.loc[first_idx:last_idx].bfill()
        return colf
    
    def gaps_fill(self, obj):
        """ 
        Function to fill gaps in df
        @param self: class
        @param obj: dataframe
        @return: dataframe without NaN
        """
        col = obj #.iloc[:, 1]
        met = 'ffill'
        df = self.fill_gap_in_col(col, met) #call function
        
        # print result
        global g_len_list
        if debug:
            #g_len_list = g_len_list*18
            #pd.options.display.max_rows = g_len_list
            #pd.options.display.max_seq_items = 20
            print(df.tail(g_len_list))
        print(f'> "NaN" gaps filled with method \'{met}\'')
        return df
    
    def time_normalize(self, obj):        
        """ 
        Function to map data of a df to a pre-specified list of timestamps
        @param self: class
        @param obj: dataframe
        @return: dataframe merged on timestamps
        """
        # read UTC:
        data = csv.DictReader(open(path+get+
            'UTC_every10mins_hc.csv'), 
            delimiter=";")
        df = pd.DataFrame(data)
        ####### take care whether timestamp is UTC-defined or not!
        df['UTC'] = pd.to_datetime(df['UTC']) #,utc=True)
        print (f'> {len(df.index)} lines in UTC')
        
        #print(obj.dtypes)
        ####### change type to (datetime64ns) from (datetime64ns, utc), if necessary
        tempcol = obj['UTC'].dt.tz_convert(None) ##.apply(lambda x: x.replace(tzinfo=None)) #or: .dt.tz_localize(None)
        obj['UTC'] = tempcol  
        #print(obj.dtypes)

        # merge on UTC:
        df_norm = pd.merge(df, obj, on='UTC', how='left')  #pd.merge(df_row, df3, left_on='id', right_on='id') if column names not the same
        
        global g_len_list
        if debug:
            print(df_norm.tail(g_len_list))
        print(f'> {len(df_norm.index)} lines time-normalized via \'left join\'')
        return df_norm
     
    def csv_export(self, obj, ROOM, VAL):
        """ 
        Function to save df to specified folder
        @param self: class
        @param obj: dataframe
        @param ROOM: part of naming
        @param VAL: second part of naming
        """
        file = path+put+str(ROOM) +'_'+ str(VAL) + '.csv' #maybe also use part: + '_cleaned_and_filled_at_' + str(date.today())
        obj.to_csv(file, sep=',', index=False, encoding='utf-8')
        print(f'> Export to \'' + file + '\' cleaned and filled successful')
   
    def db_import(self, obj):
        """ 
        Function to save df into table of QuestDB
        @param self: class
        @param obj: dataframe
        """
        print(f'> Trying to import {len(obj.index)} lines into QuestDB')
        HOST = 'localhost'
        PORT = 9009

        try:
            with Sender(HOST, PORT) as sender:
                sender.dataframe(
                        obj,
                        table_name=ROOM,  # Table name to insert into.  e.g. 'BA8'
                        #symbols=['GW_Wind'],  # Columns to be inserted as SYMBOL types.
                        at='UTC')  # Column containing the designated timestamps.
            print(f'> Storage of \'{ROOM}_{VAL}\' cleaned and filled into DB successful')
        except IngressError as e:
            sys.stderr.write(f'>Got error: {e}\n')
    
if __name__ == "__main__":
    Import()  #call Import class and there it directly jumps to __init__

> Column "Datum" or "Zeit" in csv NOT found, applying appropriate pre-import steps
> 324543 lines into DF imported
> 324520 lines in DF ok (after duplicates)
> 359580 lines in UTC
> 359580 lines time-normalized via 'left join'
> "NaN" gaps filled with method 'ffill'


C:\Users\andre\AppData\Local\Temp\ipykernel_43552\2225627592.py:142: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  obj['UTC'] = tempcol


> Export to 'S:/Andreas/FH/Technikum/BA/40_Prep/OG_W.csv' cleaned and filled successful


### Helper functions

During coding found another easy way to **create time stamps** (instead of reading them into a df out of a file).\
Yet another way would be to also specify the end date as a second input instead of "periods".

> Please be aware that the index values then have to be converted into a column in order to be used in e.g. "merge".

In [13]:
import pandas as pd
#another easy way to create time stamps
index = pd.date_range("2016-11-14 15:10:00", periods=359580, freq="10T")  # create time stamps with 10mins, is already DatetimeIndex
index

#create column with dates
dat_list = pd.DataFrame(columns = ['UTC'])
dat_list['UTC'] = index
dat_list

,UTC
0,2016-11-14 15:10:00
1,2016-11-14 15:20:00
2,2016-11-14 15:30:00
3,2016-11-14 15:40:00
4,2016-11-14 15:50:00
...,...
359575,2023-09-16 16:20:00
359576,2023-09-16 16:30:00
359577,2023-09-16 16:40:00
359578,2023-09-16 16:50:00
